Team包含一组代理、一批任务及组织代理解决任务的流程

In [1]:
# 定义互联网检索工具
import os
import api_key
os.environ['TAVILY_API_KEY']=api_key.TAVILY_API_KEY
from langchain_community.tools.tavily_search import TavilySearchResults

# 定义工具
# search = TavilySearchResults(max_results=1)

from pydantic import Field
from crewai.tools import BaseTool
class SearchTool(BaseTool):
    name: str = "Search"
    description: str = "Useful for search-based queries. Use this to find current information about markets, companies, and trends."
    search: TavilySearchResults = Field(default_factory=TavilySearchResults)

    def _run(self, query: str) -> str:
        """Execute the search query and return results"""
        try:
            self.search.max_results=1
            return self.search.run(query)
        except Exception as e:
            return f"Error performing search: {str(e)}"

In [2]:
# 定义Agent
from api_key import SERPER_API_KEY,TAVILY_API_KEY
import os
os.environ["SERPER_API_KEY"] = SERPER_API_KEY  # serper.dev API key

from crewai import  Agent
from crewai_tools import SerperDevTool
from crewai.llm import LLM

llm=LLM(model="ollama_chat/qwen2.5:latest", base_url="http://192.168.3.155:11434")
embedder_config={
        "provider": "ollama",
        "config": {
            "model": 'quentinz/bge-large-zh-v1.5:latest',
            "base_url": 'http://192.168.3.155:11434'
        }
    }

# search_tool=SerperDevTool(n_results=3,country='china')
search_tool=SearchTool()

cv_agent=Agent(
    role="计算机视觉工程师",
    goal="查找并理解计算机视觉工程师的工作内容，熟悉的代码和工具",
    backstory = "已经在计算机视觉领域工作5年，有丰富的计算机视觉经验，尤其是在工业缺陷检测方面",
    llm=llm,
    memory=True,
    verbose=True,
    tools=[search_tool],
    embedder_config=embedder_config,
    allow_delegation=False,
    max_rpm=5,
    max_execution_time=15,
)

nlp_agent=Agent(
    role="自然语言开发工程师",
    goal="查找并理解计算机视觉工程师的工作内容，熟悉的代码和工具",
    backstory = "已经在自然语言开发领域工作10年，有丰富的自然语言开发经验",
    llm=llm,
    memory=True,
    verbose=True,
    tools=[search_tool],
    embedder_config=embedder_config,
    allow_delegation=False,
    max_rpm=5,
    max_execution_time=15,
)


plan_agent=Agent(
    role="职业规划工程师",
    goal="了解不同岗位的职业需求，并为用户规划未来职业发展",
    backstory = "在大公司担任HR十年，尤其擅长为计算机领域的人做职业规划",
    llm=llm,
    memory=True,
    verbose=True,
    tools=[search_tool],
    embedder_config=embedder_config,
    allow_delegation=True,
    max_rpm=5,
    max_execution_time=30,
)

In [3]:
from crewai import Task

# 定义任务
cv_task=Task(
    description="""
    查找近期招聘计算机视觉工程师的发展趋势，岗位需求及薪资水平，
    了解那个领域的计算机视觉工程师薪资相对比较高
    """,
    expected_output="""
    输出3段文字，分别总结计算机视觉工程师的发展趋势、岗位需求、薪资
    """,
    agent=cv_agent
)

nlp_task=Task(
    description="""
    查找最近招聘计算机自然语言开发工程师的发展趋势，岗位需求及薪资水平，
    了解那个领域的计算机自然语言开发工程师薪资相对比较高
    """,
    expected_output="""
    输出3段文字，分别总结计算机自然语言开发工程师的发展趋势、岗位需求、薪资
    """,
    context=[cv_task], # 等待cv_task完成后再进行nlp_task
    agent=nlp_agent
)

write_blog_task = Task(
    description="写一篇文章，从多个层面讨论计算机视觉工程师转行到计算机自然语言开发工程师，包括技术、能力、薪资、未来发展等",
    expected_output="按层次输出文章，Markdown格式",
    context=[cv_task, nlp_task],
    agent=plan_agent
)

In [4]:
# 定义团队
from crewai import Crew

team=Crew(
    agents=[cv_agent,nlp_agent,plan_agent],
    tasks=[cv_task,nlp_task,write_blog_task],
    verbose=True,
    language='Chinese'
)

In [5]:
# 团队开始工作
result = team.kickoff()
print(result)

# Agent: 计算机视觉工程师
## Task: 
    查找近期招聘计算机视觉工程师的发展趋势，岗位需求及薪资水平，
    了解那个领域的计算机视觉工程师薪资相对比较高
    


# Agent: 计算机视觉工程师
## Using tool: Search
## Tool Input: 
"{\"query\": \"\\u8fd1\\u671f\\u8ba1\\u7b97\\u673a\\u89c6\\u89c9\\u5de5\\u7a0b\\u5e08\\u62db\\u8058\\u8d8b\\u52bf \\u5c97\\u4f4d\\u9700\\u6c42 \\u85aa\\u8d44\\u6c34\\u5e73 \\u9ad8\\u85aa\\u9886\\u57df\"}"
## Tool Output: 
[{'url': 'https://www.jobui.com/salary/quanguo-jisuanjishijiaogongchengshi/', 'content': '计算机视觉工程师工资（就业前景） 计算机视觉工程师招聘工资收入一般多少钱一个月？\r 56.1%岗位拿￥20-50K/月，年薪￥24-60W，2023年较2022持平。\r 按学历统计，大专工资￥11.4K。\r 按经验，应届生工资￥25.0K。\r 就业前景怎么样？市场需求：2024年招聘职位135个，占全国0.001%。\r 就业单位：人工智能占39.5%，软件占21%，网络占17.4%。\r 计算机视觉工程师月薪 计算机视觉工程师年薪 说明：计算机视觉工程师一个月年薪多少钱？数据统计依赖于各平台发布的公开薪酬，仅供参考。 招聘要求分析：学历本科最多 招聘行业分布：人工智能最多 你可能感兴趣的岗位 更多 历年招聘工资变化趋势 历年工资变化 就业前景（历年职位需求变化） 历年招聘职位量占比 招聘学历要求：本科最多 按学历统计 说明：薪资一般与学历正相关，一般学历越高，工资越高。计算机视觉工程师工资按学历统计，大专工资￥11.4K，想知道其他学历工资，请点击查看 招聘经验要求：1-3年最多 按经验统计 说明：工作经验是影响工资水平的重要因素，一般经验越丰富工资越高。计算机视觉工程师工资按经验统计，应届生工资￥25.0K，想知道其他经验工资，请点击查看 计